In [2]:
from tqdm import tqdm
import ujson as json
import numpy as np
import random
from IPython import embed
from collections import defaultdict, Counter
import copy
import pickle
import spacy as sp
import networkx as nx
import os
import argparse
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel, BartModel
import torch
from collections import defaultdict
import nltk
from nltk.corpus import wordnet as wn
import collections

from nltk.corpus import wordnet as wn
from copy import copy, deepcopy

import importlib
import prepro
importlib.reload(prepro)
from prepro import read_docred

BERT_DIR = '/cpfs/user/cht/cbsp/'
MAX_SEQ_LENGTH = 1024
rel2id_path = 'dataset/meta/rel2id.json'
rel2id = json.load(open(rel2id_path))
id2rel = {v:k for k,v in rel2id.items()}

# Load tokenizer and new keyword dataset

In [5]:
dev_keys_new = json.load(open('dataset/docred/dev_keys_new.json'))
kdict = pickle.load(open('dataset/docred/keywords_dict.pkl','rb'))
model_type = 'roberta-large'  # 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(BERT_DIR + model_type)
len(dev_keys_new), len(kdict.keys())

(699, 644)

# Build keyword attack dataset

In [140]:
print(docred_rel2id)

{'P6': 1, 'P17': 2, 'P19': 3, 'P20': 4, 'P22': 5, 'P25': 6, 'P26': 7, 'P27': 8, 'P30': 9, 'P31': 10, 'P35': 11, 'P36': 12, 'P37': 13, 'P39': 14, 'P40': 15, 'P50': 16, 'P54': 17, 'P57': 18, 'P58': 19, 'P69': 20, 'P86': 21, 'P102': 22, 'P108': 23, 'P112': 24, 'P118': 25, 'P123': 26, 'P127': 27, 'P131': 28, 'P136': 29, 'P137': 30, 'P140': 31, 'P150': 32, 'P155': 33, 'P156': 34, 'P159': 35, 'P161': 36, 'P162': 37, 'P166': 38, 'P170': 39, 'P171': 40, 'P172': 41, 'P175': 42, 'P176': 43, 'P178': 44, 'P179': 45, 'P190': 46, 'P194': 47, 'P205': 48, 'P206': 49, 'P241': 50, 'P264': 51, 'P272': 52, 'P276': 53, 'P279': 54, 'P355': 55, 'P361': 56, 'P364': 57, 'P400': 58, 'P403': 59, 'P449': 60, 'P463': 61, 'P488': 62, 'P495': 63, 'P527': 64, 'P551': 65, 'P569': 66, 'P570': 67, 'P571': 68, 'P576': 69, 'P577': 70, 'P580': 71, 'P582': 72, 'P585': 73, 'P607': 74, 'P674': 75, 'P676': 76, 'P706': 77, 'P710': 78, 'P737': 79, 'P740': 80, 'P749': 81, 'P800': 82, 'P807': 83, 'P840': 84, 'P937': 85, 'P1001': 8

In [142]:
file_in = 'dataset/docred/dev_keys_new.json'
i_line, pos_samples, neg_samples = 0,0,0
ori_rel_pairs = []
# mask_features is model_type specific
ori_features, mask_features, ori_anto_features, anto_features, ori_syno_features, syno_features = [],[],[],[],[],[]
mask_num, anto_num, syno_num, ori_num = 0,0,0,0

limit = False
with open(file_in, "r") as fh:
    data = json.load(fh)
if limit:
    data = data[:10]

for sample in tqdm(data, desc="Example"):
    sents = sample['sents']
    entities = sample['vertexSet']
    train_triple = {}
    htr2label = {}
    if "labels" in sample:
        for label in sample['labels']:
            evidence = label['evidence'] if 'evidence' in label else []
            r = int(docred_rel2id[label['r']])
            if (label['h'], label['t']) not in train_triple:
                train_triple[(label['h'], label['t'])] = [
                    {'relation': r, 'evidence': evidence}]
            else:
                train_triple[(label['h'], label['t'])].append(
                    {'relation': r, 'evidence': evidence})
            h,t = label['h'], label['t']
            label['h'], label['t'] = 0, 1
            htr2label[(h,t, r)] = label

    relations, hts = [], []
    for h, t in train_triple.keys():
        relation = [0] + [0] * len(docred_rel2id)
        for mention in train_triple[h, t]:
            relation[mention["relation"]] = 1
            evidence = mention["evidence"]
        relations.append(relation)
        hts.append([h, t])
        pos_samples += 1

    key_dict = defaultdict(list)
    for h,t,r,sent_id,st,ed,name in kdict[sample['title']]:
        key_dict[(h,t,r)].append((sent_id, st, ed, name))
    for h,t,r in key_dict.keys():
        if [h,t] not in hts: continue
        mask_sents, anto_sents, syno_sents = deepcopy(sents),deepcopy(sents),deepcopy(sents)
        anto_index_now, syno_index_now = 0, 0
        has_anto, has_syno = False, False

        key_dict_sort = sorted(key_dict[(h,t,r)], key=lambda x: x[0])
        for sent_id, st, ed, name in key_dict_sort:
            # do masking strategy
            for po in range(st,ed):
                tok_len = len(tokenizer.tokenize(sents[sent_id][po]))
                mask_sents[sent_id][po] = tokenizer.mask_token * tok_len

            # do wordnet synonyms and antonym replace strategy
            antonyms, synonyms = [], []
            if name == None: continue  # wrong case
            name = name.split() 
            # only consider first word
            for syn in wn.synsets(name[0]):
                for l in syn.lemmas():
                    synonyms.append(l.name())
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())

            # antonyms
            if len(antonyms) > 0:
                antonym = antonyms[0]  # first antonym
                if antonym != name:
                    assert len(antonym.split()) == 1
                    anto_sents[sent_id][st] = antonym
                    has_anto = True

            # synonyms
            if len(synonyms) > 0:
                synonym = synonyms[0]  # first antonym
                if synonym != name:
                    assert len(synonym.split()) == 1
                    syno_sents[sent_id][st] = synonym
                    has_syno = True


        ori_rel_pairs.append((entities[h][0]['name'], entities[t][0]['name'], id2rel[r]))
        ht_entities = [entities[h], entities[t]]
        ht_labels = [htr2label[(h,t,r)]]
        ht_title = f'{sample["title"]}_{h}_{t}_{r}'


        # original feature construction
        ori_dict = {
            'vertexSet': ht_entities,
            'labels': ht_labels,
            'title': ht_title,
            'sents': sents
        }
        ori_features.append(ori_dict)
        ori_num += 1

        # mask feature construction
        mask_dict = {
            'vertexSet': ht_entities,
            'labels': ht_labels,
            'title': ht_title,
            'sents': mask_sents
        }
        mask_features.append(mask_dict)
        mask_num += 1

        # replace feature construction
        if has_anto:
            anto_dict = {
                'vertexSet': ht_entities,
                'labels': ht_labels,
                'title': ht_title,
                'sents': anto_sents
            }
            anto_features.append(anto_dict)
            anto_num += 1
            ori_anto_features.append(ori_dict)

        if has_syno:
            syno_dict = {
                'vertexSet': ht_entities,
                'labels': ht_labels,
                'title': ht_title,
                'sents': syno_sents
            }
            syno_features.append(syno_dict)
            syno_num += 1
            ori_syno_features.append(ori_dict)

attack_dir = 'attack_pkl/'

json.dump(ori_features, open(attack_dir + model_type + '@ori_keyword_dev.json', 'w'))
json.dump(mask_features, open(attack_dir + model_type + '@mask_keyword_dev.json', 'w'))
json.dump(anto_features, open(attack_dir + model_type + '@anto_keyword_dev.json', 'w'))
json.dump(ori_anto_features, open(attack_dir + model_type + '@ori_anto_keyword_dev.json', 'w'))
json.dump(syno_features, open(attack_dir + model_type + '@syno_keyword_dev.json', 'w'))
json.dump(ori_syno_features, open(attack_dir + model_type + '@ori_syno_keyword_dev.json', 'w'))

print("# of documents {}.".format(i_line))
print("# of original num {}.".format(ori_num))
print("# of mask num {}.".format(mask_num))
print("# of antonym num {}.".format(anto_num))
print("# of synonym num {}. ref count {}".format(syno_num, len(ori_syno_features)))

Example: 100%|██████████| 699/699 [00:09<00:00, 75.95it/s] 


# of documents 0.
# of original num 7342.
# of mask num 7342.
# of antonym num 2002.
# of synonym num 5231. ref count 5231


# Load keyword attack dataset

In [165]:
model_type

'roberta-large'

In [4]:
attack_dir = 'attack_pkl/'
ori_path = attack_dir + model_type + '@ori_keyword_dev.json'
mask_path = attack_dir + model_type + '@mask_keyword_dev.json'
anto_path = attack_dir + model_type + '@anto_keyword_dev.json'
ori_anto_path = attack_dir + model_type + '@ori_anto_keyword_dev.json'
syno_path = attack_dir + model_type + '@syno_keyword_dev.json'
ori_syno_path = attack_dir + model_type + '@ori_syno_keyword_dev.json'

In [5]:
ori_features = read_docred(ori_path, tokenizer,ast=False)
mask_features = read_docred(mask_path, tokenizer,ast=False)
anto_features = read_docred(anto_path, tokenizer,ast=False)
ori_anto_features = read_docred(ori_anto_path, tokenizer,ast=False)
syno_features = read_docred(syno_path, tokenizer,ast=False)
ori_syno_features = read_docred(ori_syno_path, tokenizer,ast=False)

len(ori_features), len(anto_features), len(syno_features)

Example: 100%|██████████| 7342/7342 [02:53<00:00, 42.25it/s]


# of documents 7342.
# of positive examples 7342.
# of negative examples 0.


Example: 100%|██████████| 7342/7342 [02:48<00:00, 43.49it/s] 


# of documents 7342.
# of positive examples 7342.
# of negative examples 0.


Example: 100%|██████████| 2002/2002 [00:46<00:00, 43.13it/s]


...

Example: 100%|██████████| 5231/5231 [02:01<00:00, 42.94it/s]


# of documents 5231.
# of positive examples 5231.
# of negative examples 0.


Example: 100%|██████████| 5231/5231 [01:56<00:00, 45.06it/s] 

# of documents 5231.
# of positive examples 5231.
# of negative examples 0.


(7342, 2002, 5231)

In [6]:
def metric_keyword(features):
    pred = report(args, model, features)
    print(len(pred))
    spreds = sorted(pred, key=lambda x:x['title'])
    spreds = [{'title':s['title'], 'r': docred_rel2id[s['r']]} for s in spreds]
    key_res = defaultdict(list)
    for s in spreds:
        key_res[s['title']].append(s['r'])
    all_rnum = len(features)
    pos_rnum = len(key_res)
    no_rnum = all_rnum - pos_rnum
    true_rnum = 0
    for title, rl in key_res.items():
        truth = int(title.split('_')[-1])
        if truth in rl:
            true_rnum += 1

    wrong_rnum = pos_rnum - true_rnum
    return key_res, all_rnum, pos_rnum, no_rnum, true_rnum, wrong_rnum

def attack_ratio(ori_key_res, key_res, ori_features):
    # ori no rel, now rel
    all_titles = set([f['title'] for f in ori_features])
    ori_titles = set(ori_key_res.keys())
    titles = set(key_res.keys())
    no_titles = all_titles - ori_titles
    nor_r_ratio = len(no_titles & titles) / len(no_titles)
    # ori rel, now no rel
    r_nor_ratio = len((all_titles - titles) & ori_titles) / len(ori_titles)
    
    no_num,true_num,false_num = 0,0,0
    for key in ori_key_res.keys():
        if key not in key_res:
            no_num += 1
        elif len(set(key_res[key]) & set(ori_key_res[key])) > 0:
            true_num += 1
        else:
            false_num += 1
    # ori one rel, now another rel
    rel_arel_ratio = false_num / len(ori_titles)
    # ori one rel, now rel covers
    rel_srel_ratio = true_num / len(ori_titles)
    
    return r_nor_ratio, nor_r_ratio, rel_arel_ratio, rel_srel_ratio

# Load model to test

In [ ]:
import importlib
import get_model
importlib.reload(get_model)
from utils import report
from get_model import get_model

model = None
del model
torch.cuda.empty_cache()  # reload model to cuda
args, model = get_model(model_type, reset=False)

# Run keyword attack

In [41]:
from attacks import metric_keyword

# Attack ratio newer version (only consider keywords that will changed)
# roberta-large 
fs = [ori_features, mask_features, ori_anto_features, anto_features, ori_syno_features, syno_features]
key_res_list = []
for f in fs:
    key_res, all_rnum, pos_rnum, no_rnum, true_rnum, wrong_rnum = metric_keyword(f)
    key_res_list.append(key_res)
    print(all_rnum, pos_rnum, no_rnum, true_rnum, wrong_rnum)

fea_strs = ['Mask', 'Antonym', 'Synonym']
# latex format
for i in range(0, len(fs),2):
    print("& %s & %.8f & %.8f & %.8f & %.8f \\\\" % ((fea_strs[i//2],) + attack_ratio(key_res_list[i], key_res_list[i+1], fs[i]))) # output as latex table format

100%|██████████| 918/918 [01:36<00:00,  9.51it/s]


5020
7342 4566 2776 4394 172


100%|██████████| 918/918 [01:37<00:00,  9.42it/s]


4311
7342 3921 3421 3725 196


100%|██████████| 251/251 [00:28<00:00,  8.72it/s]


...

1198
2002 1135 867 1074 61


100%|██████████| 654/654 [01:11<00:00,  9.14it/s]


3452
5231 3169 2062 3050 119


100%|██████████| 654/654 [01:11<00:00,  9.12it/s]


3355
5231 3088 2143 2966 122
& Mask & 0.16513360 & 0.03926513 & 0.00503723 & 0.82982917 \\
& Antonym & 0.08422791 & 0.03286979 & 0.01156069 & 0.90421140 \\
& Synonym & 0.03849795 & 0.01988361 & 0.00126223 & 0.96023982 \\


# Entity Attack

## Entity Attack Dataset Generation

1. entity mask  
2. entity shuffle 
3. entity replacement with entity that does not exist in dataset

In [ ]:
with open('repl_ens.pkl', 'rb') as e:
    repl_ens = pickle.load(e)
docred_rel2id = json.load(open('dataset/meta/rel2id.json'))

repl_ens[:10],len(repl_ens), [r for r in repl_ens if len(r.split(' ')) > 1]

In [86]:
from itertools import groupby
from copy import deepcopy, copy

# 1. create [MASK] for all entity
# 2. entity shuffle with next entity( all mention to the first mention)
# 3. entity replaced with entity that out of distribution (NYT temporarily)

#file_in = 'dataset/docred/dev.json'
file_in = 'dataset/docred/dev_wo_overlap.json'

tag = file_in.split('/')[-1].split('.')[0]

i_line, pos_samples, neg_samples = 0, 0, 0
ori_objs, en_mask_objs, en_shuf_objs, en_repl_objs = [], [], [], []
mask_num, shuf_num, ood_num, ori_num = 0,0,0,0  
MAX_SEQ_LENGTH = 1024
MASK_TOKEN = tokenizer.mask_token

limit = False
with open(file_in, "r") as fh:
    data = json.load(fh)
if limit:
    data = data[1109:1110]

for sample in tqdm(data, desc="Example"):
    sents,labels = sample['sents'], sample['labels']
    entities = sample['vertexSet']
    title = sample['title']
    all_mentions = []

    # all entity mention and position
    for ei,e in enumerate(entities):
        for mi,m in enumerate(e):
            all_mentions.append([m['pos'][0], m['pos'][1], m['sent_id'], m['type'], m['name'], ei, mi]) # e_ind and m_ind
    all_mentions.sort(key=lambda x:(x[2],x[0]))

    # 1) entity mask 
    en_mask_sents = deepcopy(sents)
    for e in entities:
        for m in e:
            for po in range(m['pos'][0], m['pos'][1]):  # skip '*' token
                tok_len = len(tokenizer.tokenize(sents[m['sent_id']][po]))
                en_mask_sents[m['sent_id']][po] = tokenizer.mask_token * tok_len
    en_mask_feature = {
            'vertexSet': entities,
            'labels': labels,
            'title': title,
            'sents': en_mask_sents
        }
    en_mask_objs.append(en_mask_feature)

    # 2)entity shuffle (move to next entity position)
    en_shuf_sents = deepcopy(sents)
    shuf_sent_offset = defaultdict(int)
    shuf_ens = []  # entity order after shuffle/shift
    for e in entities:
        shuf_ens.append(e[0]['name'])
    shuf_ens = shuf_ens[-1:] + shuf_ens[:-1] # rotate to shuffle
    shuf_mentions = []
    for m in all_mentions:
        sent_id = m[2]
        offset = shuf_sent_offset[sent_id]
        new_en = shuf_ens[m[-2]]
        new_en = new_en.split()
        new_st, new_ed = m[0]+offset, m[0]+len(new_en)+offset

        shuf_mentions.append([new_st, new_ed, m[2], m[3], ' '.join(new_en)] + m[-2:])
        ori_len = m[1] - m[0]
        en_shuf_sents[sent_id] = en_shuf_sents[sent_id][:new_st] + new_en + en_shuf_sents[sent_id][new_st+ori_len:]
        offset += len(new_en) - ori_len
        shuf_sent_offset[sent_id] = offset

    shuf_mentions.sort(key=lambda x:x[-2])
    shuf_mens = [sorted([s for s in g], key=lambda y:y[-1]) for k, g in groupby(shuf_mentions, key = lambda x: x[-2])]  # need to groupby after sort
    shuf_entities = []
    for e in shuf_mens:
        shuf_entities.append([])
        for m in e:
            shuf_entities[-1].append({
                'name': m[4],
                'pos': [m[0], m[1]],
                'sent_id': m[2],
                'type': m[3]
            })
    en_shuf_feature = {
            'vertexSet': shuf_entities,
            'labels': labels,
            'title': title,
            'sents': en_shuf_sents
        }
    en_shuf_objs.append(en_shuf_feature)

    # 3)entity replacement (using ood entity name)
    en_repl_sents, repl_mentions = deepcopy(sents), []
    repl_sent_offset = defaultdict(int)
    repl_mentions = []
    for m in all_mentions:
        sent_id = m[2]
        offset = repl_sent_offset[sent_id]
        new_en = repl_ens[m[-2] * 10] # fetch every 10 entities
        new_en = new_en.split()
        new_st,new_ed = m[0]+offset, m[0]+len(new_en)+offset
        
        repl_mentions.append([new_st, new_ed, m[2], m[3], ' '.join(new_en)] + m[-2:])
        ori_len = m[1] - m[0]
        en_repl_sents[sent_id] = en_repl_sents[sent_id][:new_st] + new_en + en_repl_sents[sent_id][new_st+ori_len:]
        offset += len(new_en) - ori_len
        repl_sent_offset[sent_id] = offset

    repl_mentions.sort(key=lambda x: x[-2])
    repl_mens = [sorted([s for s in g], key=lambda y:y[-1]) for k, g in groupby(repl_mentions, key = lambda x: x[-2])]
    repl_entities = []
    for e in repl_mens:
        repl_entities.append([])
        for m in e:
            repl_entities[-1].append({
                'name': m[4],
                'pos': [m[0], m[1]],
                'sent_id': m[2],
                'type': m[3]
            })
    en_repl_feature = {
            'vertexSet': repl_entities,
            'labels': labels,
            'title': title,
            'sents': en_repl_sents
        }
    en_repl_objs.append(en_repl_feature)

    i_line += 1

attack_dir = 'attack_pkl/'
# json.dump(ori_features, open(attack_dir + model_type + '@ori_keyword_dev.pkl', 'wb'))
en_mask_path = attack_dir + tag + '@' + model_type + '@en_mask_dev.json'
en_shuf_path = attack_dir + tag + '@' + model_type + '@en_shuf_dev.json'
en_repl_path = attack_dir + tag + '@' + model_type + '@en_repl_dev.json'
ori_objs = json.load(open(file_in))

json.dump(en_mask_objs, open(en_mask_path, 'w'))
json.dump(en_shuf_objs, open(en_shuf_path, 'w'))
json.dump(en_repl_objs, open(en_repl_path, 'w'))
print("# of documents {}.".format(i_line))
# print("# of original num {}.".format(ori_num))
print("# of shuffle num {}.".format(len(en_shuf_features)))

Example: 100%|██████████| 66156/66156 [00:16<00:00, 3907.51it/s]


# of documents 66156.
# of shuffle num 66156.


In [74]:
idx = 51748
print(ori_objs[idx])
print()
print(en_shuf_objs[idx])
print()
print(en_mask_objs[idx])
print()
print(en_repl_objs[idx])

{'datsaset': 'scirec_test', 'title': 'scirec_test_0', 'sents': [['Recognition', 'of', 'proper', 'nouns', 'in', 'Japanese', 'text', 'has', 'been', 'studied', 'as', 'a', 'part', 'of', 'the', 'more', 'general', 'problem', 'of', 'morphological', 'analysis', 'in', 'Japanese', 'text', 'processing', '(', '[', '1', ']', '[', '2', ']', ')', '.']], 'vertexSet': [[{'sent_id': 0, 'pos': [0, 4], 'name': 'Recognition of proper nouns', 'type': 'TASK'}], [{'sent_id': 0, 'pos': [2, 4], 'name': 'proper nouns', 'type': 'OTHERSCIENTIFICTERM'}], [{'sent_id': 0, 'pos': [5, 7], 'name': 'Japanese text', 'type': 'MATERIAL'}], [{'sent_id': 0, 'pos': [19, 21], 'name': 'morphological analysis', 'type': 'TASK'}], [{'sent_id': 0, 'pos': [22, 25], 'name': 'Japanese text processing', 'type': 'TASK'}]], 'labels': [{'r': 'P361', 'h': 0, 't': 3, 'evidence': [0]}, {'r': 'P361', 'h': 1, 't': 2, 'evidence': [0]}]}

{'vertexSet': [[{'name': 'Japanese text processing', 'pos': [0, 3], 'sent_id': 0, 'type': 'TASK'}], [{'name':

## Load entity attack dataset

In [6]:
file_in = 'dataset/docred/dev_wo_overlap.json'

In [68]:
attack_dir = 'attack_pkl/'
# json.dump(ori_features, open(attack_dir + model_type + '@ori_keyword_dev.pkl', 'wb'))
tag = file_in.split('/')[-1].split('.')[0]

en_mask_path = attack_dir + tag + '@' + model_type + '@en_mask_dev.json'
en_shuf_path = attack_dir + tag + '@' + model_type + '@en_shuf_dev.json'
en_repl_path = attack_dir + tag + '@' + model_type + '@en_repl_dev.json'

# ori_json = json.load(open(file_in))
# en_mask_json = json.load(open(attack_dir + model_type + '@en_mask_dev.json'))
# en_shuf_json = json.load(open(attack_dir + model_type + '@en_shuf_dev.json'))
# en_repl_json = json.load(open(attack_dir + model_type + '@en_repl_dev.json'))

In [ ]:
ori_features = read_docred(file_in, tokenizer)
en_mask_features = read_docred(en_mask_path, tokenizer)
en_shuf_features = read_docred(en_shuf_path, tokenizer)
en_repl_features = read_docred(en_repl_path, tokenizer)
len(en_mask_features), len(en_shuf_features), len(en_repl_features)

## Run entity attack on dev_wo_overlap dataset

In [154]:
from evaluation import evaluate
## bert on dev_wo_overlap
all_feas = [ori_features, en_mask_features, en_shuf_features, en_repl_features] 
f1_outs = []
args.dev_file = 'dev_wo_overlap.json'
for fea in all_feas:
    _, f1_out = evaluate(args, model, fea, tag="dev")
    print(f1_out)
    f1_outs.append(f1_out)
en_attack_strs = ['original', 'entity mask', 'entity move', 'entity replace']
for i in range(0, len(f1_outs)):
    print(f'& {en_attack_strs[i]} & {f1_outs[i]["dev_F1"]} & {f1_outs[i]["dev_F1_ign"]} \\\\ ')

{'dev_F1': 60.76858731486493, 'dev_F1_ign': 58.95526998802616}
{'dev_F1': 6.39078051335778, 'dev_F1_ign': 6.390185517497591}
{'dev_F1': 6.084205402774398, 'dev_F1_ign': 5.689488987412854}
{'dev_F1': 14.16457286432161, 'dev_F1_ign': 14.09384602035105}
& original & 60.76858731486493 & 58.95526998802616 \\ 
& entity mask & 6.39078051335778 & 6.390185517497591 \\ 
& entity move & 6.084205402774398 & 5.689488987412854 \\ 
& entity replace & 14.16457286432161 & 14.09384602035105 \\ 


In [132]:
from evaluation import evaluate
## roberta-large original on dev_wo_overlap
all_feas = [ori_features, en_mask_features, en_shuf_features, en_repl_features] 
f1_outs = []
args.dev_file = 'dev_wo_overlap.json'
for fea in all_feas:
    _, f1_out = evaluate(args, model, fea, tag="dev")
    print(f1_out)
    f1_outs.append(f1_out)
en_attack_strs = ['original', 'entity mask', 'entity move', 'entity replace']
for i in range(0, len(f1_outs)):
    print(f'& {en_attack_strs[i]} & {f1_outs[i]["dev_F1"]} & {f1_outs[i]["dev_F1_ign"]} \\\\ ')

{'dev_F1': 61.62089105565478, 'dev_F1_ign': 59.744852760161}
{'dev_F1': 27.28648648648648, 'dev_F1_ign': 27.083648897895486}
{'dev_F1': 7.352144876962879, 'dev_F1_ign': 6.96639315184737}
{'dev_F1': 17.497299799413675, 'dev_F1_ign': 17.406135449708817}
& original & 61.62089105565478 & 59.744852760161 \\ 
& entity mask & 27.28648648648648 & 27.083648897895486 \\ 
& entity move & 7.352144876962879 & 6.96639315184737 \\ 
& entity replace & 17.497299799413675 & 17.406135449708817 \\ 
